# Main Problem

In [51]:
import json

In [69]:
def get_transport_matrix():
    BASE_PATH = "../data/routeData/"
    daytimes = [("morning", 8), ("midday", 12), ("evening", 16), ("night", 20)]
    transport_matrix = {}

    for time_in_day, hour in daytimes:
        filepath = BASE_PATH + f"route_matrix_{time_in_day}.json"

        with open(filepath, 'r') as f:
            route_matrix = json.load(f)

        for route in route_matrix["routes"]:
            this_route = route_matrix["routes"][route]
            origin = this_route["origin_name"]
            destination = this_route["destination_name"]
            transport_matrix[(origin, destination, hour)] = {
                "transit": {
                    "duration": this_route["transit"]["duration_minutes"],
                    "price": this_route["transit"]["fare_sgd"],
                },
                "drive": {
                    "duration": this_route["drive"]["duration_minutes"],
                    "price": this_route["drive"]["fare_sgd"],
                }
            }
    
    return transport_matrix


def get_all_locations():
    BASE_PATH = "../data/routeData/"

    with open(BASE_PATH + "route_matrix_morning.json", 'r') as f:
        route_matrix = json.load(f)
    
    locations = [route_matrix["locations"][location_id] for location_id in route_matrix["locations"]]
    return locations


def get_transport_hour(transport_time):
    # because the transport_matrix is only bracketed to 4 groups, we find the earliest it happens
    brackets = [8, 12, 16, 20]
    transport_hour = transport_time // 60

    for bracket in reversed(brackets):
        if transport_hour >= bracket:
            return bracket

    return brackets[-1] # from 8 PM to 8 AM next day


In [70]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem


class TravelItineraryProblem(ElementwiseProblem):

    def __init__(self, budget, locations, transport_matrix):
        # constants
        self.NUM_DAYS = 3
        self.HOTEL_COST = 50
        self.START_TIME = 9 * 60 # everyday starts from 9 AM
        self.HARD_LIMIT_END_TIME = 22 * 60 # everyday MUST return back to hotel by 10 PM

        # hard limit money spent
        self.budget = budget
        # list of destinations
        self.locations = locations
        self.num_locations = len(locations)
        self.num_attractions = len([loc for loc in self.locations if loc["type"] == "attraction"])
        self.num_hawkers = len([loc for loc in self.locations if loc["type"] == "hawker"])
        self.num_hotels = 1 # always 1

        # transportation option prices and durations. The size MUST BE num_dest * num_dest, for 24 hours
        self.transport_types = ["transit", "drive"]
        self.num_transport_types = len(self.transport_types)
        self.transport_matrix = transport_matrix

        # x_ijkl = BINARY VAR, if the route goes in day i, use transport type j, from location k, to location l
        self.x_shape = self.NUM_DAYS * self.num_transport_types * self.num_locations * self.num_locations
        # u_ik  = CONTINUOUS VAR, tracking FINISH time of the person in day i, location k
        self.u_shape = self.NUM_DAYS * self.num_locations

        num_vars = self.x_shape + self.u_shape

        lower_bound = np.concatenate([np.zeros(self.x_shape), np.full(self.u_shape, self.START_TIME)])
        upper_bound = np.concatenate([np.ones(self.x_shape), np.full(self.u_shape, self.HARD_LIMIT_END_TIME)])

        # inequality constraints
        #   - for every attraction, must be a source at least once
        #   - vice versa, for every attraction, must be a destination at least once
        #   - if source k to destination l chosen, make sure the time in var u is correct
        #       - from hotel, it can go to N-1 destinations (exclude hotel)
        #       - from any other destinations, it can go to N-2 destinations (exclude itself AND hotel)
        #   - every day, must go to hawker at least twice
        #   - if public transport is chosen, taxi can't be chosen
        #   - make sure everything is within budget
        num_inequality_constraints = \
            self.num_attractions + \
            self.num_attractions + \
            self.NUM_DAYS * self.num_transport_types * (self.num_locations - 1) + \
            self.NUM_DAYS * self.num_transport_types * (self.num_locations - 1) * (self.num_locations - 2) + \
            self.NUM_DAYS + \
            self.NUM_DAYS * self.num_locations * self.num_locations + \
            1

        # equality constraints
        #   - everyday, hotel must be starting
        #   - everyday, for every attraction, if selected as source, must be selected as destination
        #   - everyday, must return to hotel from last destination
        num_equality_constraints = \
            self.NUM_DAYS + \
            self.NUM_DAYS + \
            self.NUM_DAYS

        super().__init__(
            n_var=num_vars,
            n_obj=3,
            n_ieq_constr=num_inequality_constraints,
            n_eq_constr=num_equality_constraints,
            xl=lower_bound,
            xu=upper_bound,
        )

    def _evaluate(self, x, out, *args, **kwargs):
        # they're ensured to be integers
        x_var = x[:self.x_shape].reshape(self.NUM_DAYS, self.num_transport_types, self.num_locations, self.num_locations)
        u_var = x[self.x_shape:].reshape(self.NUM_DAYS, self.num_locations)

        # initialize constraints
        # equality constraints
        out["H"] = []
        # inequality constraints
        out["G"] = []

        total_cost = self.NUM_DAYS * self.HOTEL_COST  # Cost starts with hotel cost for number of nights
        total_travel_time = 0
        total_satisfaction = 0

        # for every attraction, must be a source at most once
        # for every attraction, must be a destination at most once
        # NOTE that this doesn't apply to hawkers
        for k in range(self.num_locations):
            if self.locations[k]["type"] == "attraction":
                out["G"].append(np.sum(x_var[:, :, k, :]) - 1)
                out["G"].append(np.sum(x_var[:, :, :, k]) - 1)

        for i in range(self.NUM_DAYS):
            # u_var[i, 0] must be the smallest of u_var[i]
            out["H"].append(np.min(u_var[i, :]) - u_var[i, 0])

            # every day, for every attraction, if it's selected as source, it must be selected as destination
            out["H"].append(np.sum(x_var[i, :, :, k]) - np.sum(x_var[i, :, k, :]))

            for j in range(self.num_transport_types):
                for k in range(self.num_locations):
                    for l in range(1, self.num_locations): # NOTE here that hotel (index 0) isn't included in destination. It will be computed later
                        if k == l: continue
                        # every day, if the route is chosen, the time of finishing in this place is this

                        # if chosen, then time_should_finish_l must be time_finish_l, otherwise, don't matter (just put 0)
                        # you should finish attraction l at:
                        #   - time to finish k + 
                        #   - time to transport from k to l, using transport method j +
                        #   - time to play at l
                        time_finish_l = u_var[i, l]
                        transport_hour = get_transport_hour(u_var[i, k])
                        transport_value = self.transport_matrix[(self.locations[k]["name"], self.locations[l]["name"], transport_hour)][self.transport_types[j]]
                        time_should_finish_l = u_var[i, k] + transport_value["duration"] + self.locations[l]["duration"]
                        # if x_var[i, j, k, l] is not chosen, then this constraint don't matter
                        out["G"].append(x_var[i, j, k, l] * time_finish_l - time_should_finish_l)
                        
                        # append to total travel time spent
                        if x_var[i, j, k, l] == 1:
                            # calculate the travel
                            total_travel_time += transport_value["duration"]
                            total_cost += transport_value["price"]

                            # calculate cost and satisfaction, based on what they come to
                            # Note: not counting anything for hotel.
                            if self.locations[l]["type"] == "attraction":
                                total_cost += self.locations[l]["entrance_fee"]
                                total_satisfaction += self.locations[l]["satisfaction"]
                            elif self.locations[l]["type"] == "hawker":
                                total_cost += 10 # ASSUME eating in a hawker is ALWAYS $10
                                total_satisfaction += self.locations[l]["rating"]

            # from last place, return to hotel.
            last_place = np.argmax(u_var[i, :])
            # IF last_place is already the hotel, then get the second last
            if last_place == 0:
                last_place = np.argsort(u_var[i, :])[-2]
            latest_hour = get_transport_hour(u_var[i, last_place])
            # MUST go back to hotel at the end of the day
            out["H"].append(np.sum(x_var[i, :, last_place, 0]) - 1)
            # go back to hotel
            # choose whether to use which transport type
            for j in range(self.num_transport_types):
                if x_var[i, j, last_place, 0]:
                    # pull from google maps to get the transport details if the decision is here.
                    gohome_value = self.transport_matrix[(self.locations[last_place]["name"], self.locations[0]["name"], latest_hour)][self.transport_types[j]]
                    total_travel_time += gohome_value["duration"]
                    total_cost += gohome_value["price"]
                    break

        for i in range(self.NUM_DAYS):
            hawker_sum = 0
            for k in range(self.num_locations):
                if self.locations[k]["type"] == "hawker":
                    hawker_sum += np.sum(x_var[i, :, k, :])

            # every day, must go to hawkers at least twice (lunch & dinner. Can go more times if they want to)
            out["G"].append(2 - hawker_sum)

        for i in range(self.NUM_DAYS):
            for k in range(self.num_locations):
                for l in range(self.num_locations):
                    # for every day, if public transportation is chosen, taxi can't be chosen, and vice versa
                    # (sum j in transport_types x_ijkl <= 1) for all days, for all sources, for all destinations
                    out["G"].append(np.sum(x_var[i, :, k, l]) - 1)

        # finally, make sure everything is within budget
        out["G"].append(self.budget - total_cost)

        # <ADD ADDITIONAL CONSTRAINTS HERE>

        # objectives
        out["F"] = [total_cost, total_travel_time, -total_satisfaction]

# TODO:
#   - fix the constraints..., for some reason no feasible solution
#   - make sure the hawker is visited DURING LUNCHTIME and DINNERTIME
#       - the "twice-a-day formulation" is there already, but not at the correct time


In [71]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.optimize import minimize
from pymoo.termination import get_termination

In [72]:
all_locations = get_all_locations()

# for all locations, get satisfaction and rating
for loc in all_locations:
    if loc["type"] == "hawker":
        loc["rating"] = np.random.uniform(0, 5)
        loc["avg_food_price"] = np.random.uniform(5, 15)
        loc["duration"] = 60 # just standardize 60 mins
    elif loc["type"] == "attraction":
        loc["satisfaction"] = np.random.uniform(0, 10)
        loc["entrance_fee"] = np.random.uniform(5, 100)
        loc["duration"] = np.random.randint(30, 90)

dummy_hotel = {
    "type": "hotel",
    "name": "DUMMY HOTEL",
    "lat": 1.2852044,
    "lng": 103.8610313,
}

In [73]:
transport_matrix = get_transport_matrix()
# add dummy hotel to transport_matrix
for loc in all_locations:
    time_brackets = [8, 12, 16, 20]
    for time_ in time_brackets:
        transport_matrix[(dummy_hotel["name"], loc["name"], time_)] = {
            "transit": {
                "duration": 50,
                "price": 1.93,
            },
            "drive": {
                "duration": 20,
                "price": 10.1,
            }
        }
        transport_matrix[(loc["name"], dummy_hotel["name"], time_)] = {
            "transit": {
                "duration": 50,
                "price": 1.93,
            },
            "drive": {
                "duration": 20,
                "price": 10.1,
            }
        }

locations = [dummy_hotel] + all_locations

In [74]:
# make the problemset and solve it.
problem = TravelItineraryProblem(
    budget=300,
    locations=locations,
    transport_matrix=transport_matrix,
)

algorithm = NSGA2(
    pop_size=100,
    sampling=IntegerRandomSampling(),
    crossover=TwoPointCrossover(),
    mutation=BitflipMutation(),
    eliminate_duplicates=True
)

termination = get_termination("n_gen", 200)

res = minimize(
    problem,
    algorithm,
    termination,
    seed=1,
    save_history=True,
    verbose=True
)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))


n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      1 |  1.597622E+05 |  2.278955E+05 |             - |             -
     2 |      200 |      1 |  1.576405E+05 |  2.088725E+05 |             - |             -
     3 |      300 |      1 |  1.528373E+05 |  1.938066E+05 |             - |             -
     4 |      400 |      1 |  1.525001E+05 |  1.783802E+05 |             - |             -
     5 |      500 |      1 |  1.525001E+05 |  1.647215E+05 |             - |             -
     6 |      600 |      1 |  1.514591E+05 |  1.578867E+05 |             - |             -
     7 |      700 |      1 |  1.502462E+05 |  1.550911E+05 |             - |             -
     8 |      800 |      1 |  1.455867E+05 |  1.523466E+05 |             - |             -
     9 |      900 |      1 |  1.453241E+05 |  1.504309E+05 |             - |             -
    10 |     1000 |      1 |  1.438310E+05 |  1.485743E+05 |             - |             -

In [76]:
res.X